<a href="https://colab.research.google.com/github/nmq443/cognitive-science-final-project/blob/quang-branch/torcheeg_atcnet-with_preprocessed_data_cwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# if run on colab
"""
!pip install torcheeg
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
"""

"\n!pip install torcheeg\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n"

In [2]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [3]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [4]:
# if run on kaggle
root_data_path = '/kaggle/input/BCICIV-2a-mat'
# if run on colab
# root_data_path = '/content/drive/MyDrive/BCICIV-2a-mat'

# if run on local machine
# root_data_path = './BCICIV-2a-mat/'

In [5]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.MinMaxNormalize(),
        transforms.ToTensor(),
        transforms.RandomNoise(p=0.3),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-22 10:09:02] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-22 10:09:02] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/18 [00:00<?, ?it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 1it [00:00,  1.00it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 10it [00:01, 12.10it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 19it [00:01, 23.63it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 28it [00:01, 34.99it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 37it [00:01, 44.93it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 46it [00:01, 53.95it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 55it [00:01, 61.27it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat

In [6]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04T_0        A04         0       T     A04    1   
1         2254    4004    A04T_1        A04         1       T     A04    1   
2         4172    5922    A04T_2        A04         2       T     A04    1   
3         6124    7874    A04T_3        A04         3       T     A04    1   
4         8132    9882    A04T_4        A04         4       T     A04    1   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A04E_283        A04        43       E     A04    8   
5180     88657   90407  A04E_284        A04        44       E     A04    8   
5181     90585   92335  A04E_285        A04        45       E     A04    8   
5182     92699   94449  A04E_286        A04        46       E     A04    8   
5183     94758   96508  A04E_287        A04        47       E     A04    8   

      label  _record_id  
0         4   _recor

In [7]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [8]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=8
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=8
    )

    model = EEGNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )

    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader,
        max_epochs=200,
        default_root_dir=f'./examples_pipeline/eegnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-22 10:09:49] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-22 10:09:49] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2024-05-22 10:09:57.247211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 10:09:57.247368: E external/loca

Training: |          | 0/? [00:00<?, ?it/s]

[2024-05-22 10:10:11] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.408 train_accuracy: 0.253 

[2024-05-22 10:10:14] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.398 train_accuracy: 0.262 

[2024-05-22 10:10:18] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.397 train_accuracy: 0.265 

[2024-05-22 10:10:22] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.393 train_accuracy: 0.277 

[2024-05-22 10:10:26] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.393 train_accuracy: 0.267 

[2024-05-22 10:10:29] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.386 train_accuracy: 0.290 

[2024-05-22 10:10:33] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.388 train_accuracy: 0.287 

[2024-05-22 10:10:37] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.379 train_accuracy: 0.289 

[2024-05-22 10:10:41] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.371 train_accuracy: 0.312 

[2024-05-22 10:10:45] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.373 train_accuracy: 0.311 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-22 10:22:45] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.240 test_accuracy: 0.463 



...

[2024-05-22 12:11:25] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.407 train_accuracy: 0.252 

[2024-05-22 12:11:29] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.395 train_accuracy: 0.271 

[2024-05-22 12:11:33] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.393 train_accuracy: 0.269 

[2024-05-22 12:11:37] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.391 train_accuracy: 0.281 

[2024-05-22 12:11:41] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.385 train_accuracy: 0.295 

[2024-05-22 12:11:46] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.379 train_accuracy: 0.305 

[2024-05-22 12:11:50] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.377 train_accuracy: 0.311 

[2024-05-22 12:11:54] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.372 train_accuracy: 0.320 

[2024-05-22 12:11:58] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.364 train_accuracy: 0.317 

[2024-05-22 12:12:02] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.362 train_accuracy: 0.311 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-22 12:24:57] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.217 test_accuracy: 0.431 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.4305555522441864     │
│         test_loss         │    1.2174174785614014     │
└───────────────────────────┴───────────────────────────┘

Fold 9 test accuracy:  0.4306


In [9]:
!mkdir weights

In [10]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))